In [32]:
import sqlite3
import numpy as np
import time
from matplotlib import pyplot as plt

#수집시간(5초)
COLLECTION_TIME = 5000

In [2]:
floor_db = 'BlackBox/BlackBox_floor.db'
floor_conn = sqlite3.connect(floor_db)
cur = floor_conn.cursor()

In [3]:
def get_baro_list(rows):
    baro_list = list()
    prev_time = -1e8
    
    for row in rows:
        element = {
        'timeMillis': row[1],
        'timeNanos': row[2],
        'value': row[3]
    }
    
        if (element['timeMillis'] - prev_time) < COLLECTION_TIME:
            baro_list[-1].append(element)
        else:
            baro_list.append([element])
            prev_time = element['timeMillis']
    
    return baro_list

In [4]:
def get_fingerprint_list(rows):
    fp_dict = dict()
    prev_time = -1e8
    
    for row in rows:
        element = {
            'timeMicros': row[1],
            'ssid': row[2],
            'bssid': row[3],
            'freq': row[4],
            'rss': row[5]
        }
        wifi_time = row[6]
        
        try:
            fp_dict[wifi_time].append(element)
        except KeyError:
            fp_dict[wifi_time] = [element]
    
    
    fp_list = [{'logTime': t, 'data': fp_dict[t]} for t in sorted(fp_dict)]
    return fp_list

In [5]:
#data fetch
cur.execute("select * from Rss where id >= 280 and id < 2436")
rows_8th_floor = cur.fetchall()

cur.execute("select * from Rss where id >= 2436")
rows_2nd_floor = cur.fetchall()

fp_list_confirmed = get_fingerprint_list(rows_8th_floor)
fp_list_user = get_fingerprint_list(rows_2nd_floor)

In [54]:
for comfirmed, user in zip(fp_list_confirmed, fp_list_user):
    confirmed_bssids = [data['bssid'] for data in comfirmed['data']]
    user_bssids = [data['bssid'] for data in user['data']]
    
    intersection = list(set(confirmed_bssids) & set(user_bssids))
    
    #in case there is no common 
    if not intersection:
        continue
    
        
    

['70:5d:cc:93:2a:84']
['00:24:6c:7a:8b:20', '90:9f:33:59:19:46', '00:24:6c:7d:f5:d0', '00:24:6c:7d:f5:d1']
['00:24:6c:7d:f5:d1', '90:9f:33:59:19:46', '00:24:6c:7d:fe:f0']
['88:36:6c:45:40:fe']
['00:24:6c:7d:f5:d1', '90:9f:33:59:19:46', 'b0:c7:45:75:c9:1f']
['00:24:6c:7d:f5:d1']
['00:24:6c:7d:f5:d1']
['00:24:6c:7d:f5:11']


In [21]:
confirmed_bssid = {}
user_bssid = {}
for confirmed, user in zip(fp_list_confirmed, fp_list_user):
    confirmed_bssid[confirmed['logTime']] = [x['bssid'] for x in confirmed['data']]
    user_bssid[user['logTime']] = [x['bssid'] for x in user['data']]

{1602593272863: ['90:9f:33:65:8b:5e',
  '90:9f:33:66:8b:5e',
  '00:24:6c:7e:02:41',
  '00:24:6c:7e:02:40'],
 1602593280926: ['90:9f:33:65:8b:5e',
  '90:9f:33:66:8b:5e',
  '00:24:6c:7e:01:71',
  '1c:3b:f3:da:45:ca',
  '00:24:6c:7e:01:70',
  '1c:3b:f3:da:45:cb',
  '00:24:6c:7d:f8:91',
  '00:24:6c:7d:f8:90',
  '00:24:6c:7d:ef:60',
  '90:9f:33:65:85:d4',
  '3e:73:28:ce:73:11',
  '00:24:6c:7e:02:40',
  '00:24:6c:7e:02:41',
  '24:6f:28:28:2b:2d',
  '88:36:6c:e1:44:1c',
  '00:24:6c:7d:ef:61',
  '24:6f:28:28:2b:2d',
  '00:24:6c:7e:00:90',
  '00:24:6c:7e:00:91',
  '72:5d:cc:cf:a7:90',
  '70:5d:cc:06:5a:96',
  '54:a0:50:d8:7c:80',
  '24:6f:28:28:2b:89',
  '00:26:66:27:42:f2',
  '90:9f:33:66:85:d4',
  '00:24:6c:7d:f2:21',
  '00:24:6c:7d:ef:70',
  '00:24:6c:7d:ef:71',
  '00:24:6c:7d:f1:71',
  '00:24:6c:7d:f2:20',
  '00:24:6c:7e:00:10',
  '88:36:6c:e1:44:1e',
  '00:24:6c:7d:f1:70',
  '00:24:6c:7d:f6:30',
  '00:24:6c:7d:f6:31',
  '00:26:66:83:e6:10',
  '00:24:6c:7e:00:11',
  '00:24:6c:7d:fe:30',
  '

In [37]:
N = min(len(user_bssid), len(confirmed_bssid))

for con_bssid, usr_bssid in zip(confirmed_bssid.values(), user_bssid.values()):
    print(len(list(set(con_bssid) & set(usr_bssid))))
    

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
4
3
0
1
0
3
0
0
0
1
0
1
0
1
0


In [ ]:
floor_conn.close()